In [1]:
import sys
sys.path[0]+='/..'
from dataset import GraphDataset
import torch
import torch_geometric
import os 
from dataset import GraphDataset
# from model.gcn_gat import GNNPolicy
from model.gcn import GNNPolicy
from readdataUC import UC
import pandas as pd
from scipy.sparse import coo_matrix
from torch.optim.lr_scheduler import CosineAnnealingLR
import time

In [2]:
# DEVICE = torch.device("cpu")
DEVICE = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
#set folder
model_save_path = '../model_save/1bin_50_pretrain_routine_gat/'
log_save_path = "../model_save/1bin_50_log_routine_gat/"
if not os.path.isdir(model_save_path):
    os.mkdir(model_save_path)
if not os.path.isdir(log_save_path):
    os.mkdir(log_save_path)
log_file = open(f'{log_save_path}train.log', 'ab')

In [3]:
LEARNING_RATE = 1e-5
NB_EPOCHS =9999
BATCH_SIZE = 1
NUM_WORKERS = 0
ETA_MIN=1e-6

In [4]:
# name = '10_std'
name = '50_0_1_w'
train_data = GraphDataset(f'../datasets/{name}/1bin/train/')
train_iter = torch_geometric.loader.DataLoader(
    train_data, batch_size = BATCH_SIZE, shuffle = True,
    num_workers = NUM_WORKERS, pin_memory=True)
valid_data = GraphDataset(f'../datasets/{name}/1bin/valid/')
valid_iter = torch_geometric.loader.DataLoader(
    valid_data, batch_size = BATCH_SIZE, shuffle = False,
    num_workers = NUM_WORKERS)

In [5]:
PredictModel = GNNPolicy().to(DEVICE)
# if os.path.exists(model_save_path+'model_best.pth'):
#     state_dict = torch.load(model_save_path+'model_best.pth')
#     PredictModel.load_state_dict(state_dict)
#     print("载入模型成功")


In [6]:


file = '../UC_AF/'+name+'.mod'

dataUC=UC(file)

N = dataUC.NumThermal
T = dataUC.HorizonLen
source_node=[]
target_node=[]
for i in range(N):
    for t in range(T):
        j=i*T+t
        source_node.append(j)
        target_node.append(j)
        source_node.append(j+N*T)
        target_node.append(j)
        source_node.append(j+2*N*T)
        target_node.append(j)
        # source_node.append(j+3*N*T)
        # target_node.append(j)
        # source_node.append(j)
        # target_node.append(j+4*N*T)
        # source_node.append(j)
        # target_node.append(j+5*N*T)


        
        for ii in range(N):
            if ii != i:
                source_node.append(j)
                target_node.append(ii*T+t)
        for tt in range(T):
            if tt != t:
                source_node.append(j)
                target_node.append(i*T+tt)
edge_index=torch.tensor([source_node,target_node], dtype=torch.long).to(DEVICE)
print(len(source_node))

90000


In [7]:
def train(predict, data_loader, optimizer=None):
    """
    This function will process a whole epoch of training or validation, depending on whether an optimizer is provided.
    """
    loss_fn = torch.nn.BCELoss()

    if optimizer:
        predict.train()
    else:
        predict.eval()
    mean_loss = 0
    n_samples_processed = 0
    with torch.set_grad_enabled(optimizer is not None):
        for step, batch in enumerate(data_loader):
            batch = batch.to(DEVICE)
            weighted_sol = batch.weighted_sol
            #predict the binary distribution, BD
            BD = predict(
                batch.constraint_features,
                batch.edge_index,
                batch.edge_attr,
                batch.variable_features,
                # edge_index
            )
            # break
            print(batch.variable_features.shape)
            break
            # compute loss
            
    #         presol = BD[batch.b_vars==1]
    #         presol = presol.sigmoid()
    #         loss = loss_fn(presol, weighted_sol)

    #         if optimizer is not None:
    #             optimizer.zero_grad()
    #             loss.backward()
    #             optimizer.step()
    #         mean_loss += loss.item()
    #         n_samples_processed += 1
    # mean_loss /= n_samples_processed

    return 0


In [8]:
optimizer = torch.optim.Adam(PredictModel.parameters(), lr=LEARNING_RATE)
scheduler = CosineAnnealingLR(optimizer, T_max=NB_EPOCHS, eta_min=ETA_MIN)
best_val_loss = 0.1689965840328384

In [9]:
for epoch in range(1):
    begin=time.time()
    train_loss = train(PredictModel, train_iter, optimizer)
    print(f"Epoch {epoch} Train loss: {train_loss:0.7f}",end='\t')
    valid_loss = train(PredictModel, valid_iter, None)
    print(f"Epoch {epoch} Valid loss: {valid_loss:0.7f}")
    if valid_loss<best_val_loss:
        best_val_loss = valid_loss
        print(best_val_loss)
        torch.save(PredictModel.state_dict(),model_save_path+'model_best.pth')
    torch.save(PredictModel.state_dict(), model_save_path+'model_last.pth')
    date_str = time.strftime('%Y-%m-%d  %H:%M:%S', time.localtime())
    st = f'@epoch{epoch}   Train loss:{train_loss}   Valid loss:{valid_loss}    TIME:{time.time()-begin}    DONE_TIME:{date_str}\n'
    log_file.write(st.encode())
    log_file.flush()
    scheduler.step()
print('done')

torch.Size([3600, 6])
Epoch 0 Train loss: 0.0000000	torch.Size([3600, 6])
Epoch 0 Valid loss: 0.0000000
0
done


/home/peilun/miniconda3/envs/gat/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
